In [3]:
import langchain
print(langchain.__version__)

0.3.27


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

### Testing LLm

In [5]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o-mini")
llm_response=llm.invoke("Tell bangladesh national anthem")

llm_response

AIMessage(content='The national anthem of Bangladesh is "Amar Sonar Bangla," written by the famous poet Rabindranath Tagore. The anthem expresses the beauty and spirit of Bangladesh and its people\'s love for their homeland. Here are the opening lines of the anthem in Bengali:\n\nআমার সোনার বাংলা, আমি তোমায় ভালোবাসি।\n  \nIf you would like a translation or a summary of the themes in the anthem, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 12, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CCI18FISU6xBWHCUBNU592fcKPoiS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c9e3c8bd-6098-480c-9c25-2321f

### Parsing output 
<sub>Here we can parse the output. it looks like we can only show the exact reply that we get from the LLM</sub>


In [7]:
from langchain_core.output_parsers import StrOutputParser

outputParser = StrOutputParser()
outputParser.invoke(llm_response)

'The national anthem of Bangladesh is "Amar Sonar Bangla," written by the famous poet Rabindranath Tagore. The anthem expresses the beauty and spirit of Bangladesh and its people\'s love for their homeland. Here are the opening lines of the anthem in Bengali:\n\nআমার সোনার বাংলা, আমি তোমায় ভালোবাসি।\n  \nIf you would like a translation or a summary of the themes in the anthem, feel free to ask!'

### Simple Chain
<sub>Here we are making a simple chain so that instead of calling one by one we can directly invoke this chain and get the answer</sub>

In [8]:
chain = llm | outputParser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!'

### Structured Output

<sub>Here we can get structure output from here</sub>

In [9]:
from typing import List
from pydantic import BaseModel, Field

class ProductReview(BaseModel):
    product_name: str = Field(description="Name and model of the product (car, bike, phone, etc.)")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects of the product")
    cons: List[str] = Field(description="List of negative aspects of the product")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just took the new Yamaha R15 V4 for a spin and it blew me away! The handling feels razor sharp,
and the sporty design definitely turns heads on the road. Acceleration is smooth, and the bike
holds its own really well at higher speeds. Fuel efficiency is also surprisingly good for a
sports bike — I managed to get around 40 km/l in city traffic.

On the flip side, the seating position can get uncomfortable on longer rides — it’s built for
sport, not comfort. Also, the pillion seat is pretty cramped, so not ideal for two-up riding.
The price is a bit on the higher side compared to competitors too.

Overall, I’d give it a 4.3 out of 5. Fantastic choice if you want a stylish, performance-focused
bike, but not the best for touring or daily long commutes.
"""


structured_llm = llm.with_structured_output(ProductReview)
structure_output = structured_llm.invoke(review_text)
structure_output


ProductReview(product_name='Yamaha R15 V4', rating=4.3, pros=['Sharp handling', 'Sporty and eye-catching design', 'Smooth acceleration', 'Good fuel efficiency (around 40 km/l in city traffic)'], cons=['Uncomfortable seating position for longer rides', 'Cramped pillion seat', 'Higher price compared to competitors'], summary='The Yamaha R15 V4 impresses with its performance and design but may not be ideal for long rides.')

In [10]:
structure_output.rating

4.3

### Prompt Template
<sub>typically msg could be dynamic. so we need some flexibilty to set any msg. so we can use the prompt template</sub>